# Objectif

Récuperer le fichier:
`https://github.com/VPerrollaz/immobilier/blob/master/donnees/brute.json`

puis transformer le en un dataset permettant de faire de l'apprentissage.

# Récupération fichier

In [1]:
from requests import get

In [2]:
ADRESSE = "https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/brute.json"

In [3]:
requete = get(ADRESSE)
requete.status_code

200

In [4]:
requete.reason

'OK'

In [5]:
contenu = requete.text

In [6]:
type(contenu)

str

In [7]:
contenu[:1000]

'{"id": "annonce-138905473-376235", "genre": "Appartement", "prix": "374 400 \\u20ac", "pcs": "3 p 2 ch 90 m\\u00b2", "desc": "Appartement type 3 - TOURS CATH\\u00c9DRALE TOURS CATH\\u00c9DRALE: Ravissant appartement type 3 d\'environ 90 m\\u00b2 dans r\\u00e9sidence de standing (~50 lots) avec cave et parking. Entr\\u00e9e,...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm"}\n{"id": "annonce-140620177-376235", "genre": "Appartement", "prix": "499 200 \\u20ac", "pcs": "5 p 4 ch 146,27 m\\u00b2", "desc": "TOURS HYPERCENTRE - Appartement TOURS HYPERCENTRE: Superbe appartement de 146,37 m\\u00b2 (LOI CARREZ) dans un immeuble de caract\\u00e8re avec ascenseur. Entr\\u00e9e, s\\u00e9jour SUD, vaste...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/centre/140620177.htm"}\n{"id": "annonce-140620179-376235", "genre": "Appartement", "prix": "499 200 \\u20ac", "pcs

In [8]:
contenu[-1000:]

'es avec jardin ou appartements du 2 au 3 pi\\u00e8ces: faites votre choix - Exceptionnel: une cuisine Blanchet d\'Huismes am\\u00e9nag\\u00e9e et install\\u00e9e dans...\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.selogerneuf.com/annonces/achat/maison/tours-37/138636083/#?cmp=INTSL_ListToDetail"}\n{"id": "annonce-137873909-139524", "genre": "Boutique", "prix": "195 000 \\u20ac", "pcs": "173 m\\u00b2", "desc": "Place Saint \\u00c9loi, Local commercial de 173m\\u00b2 avec acc\\u00e8s PMR. Il comprend des espaces bureaux en rez-de-chauss\\u00e9e de 92m\\u00b2 environ et 79m\\u00b2 \\u00e0 l\'\\u00e9tage. Belle fa\\u00e7ade.\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/boutique/tours-37/137873909.htm"}\n{"id": "annonce-136457803-733454", "genre": "Parking", "prix": "15 000 \\u20ac", "pcs": "13 m\\u00b2", "desc": "GRAMMONT.PLACE DE LA LIBERT\\u00c9. Garage en sous-sol.\\n\\u00c7a m\'int\\u00e9resse", "lien": "https://www.seloger.com/annonces/acha

# Décodage du json

Délicat car construction à la main sans librairie de sérialisation (i.e. `pyserde`)!

In [9]:
import json

In [10]:
json.loads(contenu)

JSONDecodeError: Extra data: line 2 column 1 (char 445)

In [11]:
contenu[440:450]

'tm"}\n{"id"'

In [12]:
print(contenu[:450])

{"id": "annonce-138905473-376235", "genre": "Appartement", "prix": "374 400 \u20ac", "pcs": "3 p 2 ch 90 m\u00b2", "desc": "Appartement type 3 - TOURS CATH\u00c9DRALE TOURS CATH\u00c9DRALE: Ravissant appartement type 3 d'environ 90 m\u00b2 dans r\u00e9sidence de standing (~50 lots) avec cave et parking. Entr\u00e9e,...\n\u00c7a m'int\u00e9resse", "lien": "https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm"}
{"id"


On voit qu'on a juste une juxtaposition d'annonces représentées par des dictionnaires.
Il faut rajouter les virgules et les crochets pour transformer le contenu en une liste de dictionnaires.

**REMARQUE** le `\\u00b2` représente 6 caractères alors que `\u00b2` représente un caractère via sa numérotation unicode!

In [13]:
print("\\u00b2")

\u00b2


In [14]:
print("\u00b2")

²


In [15]:
annonces = json.loads("[" + contenu[:-1].replace("\n", ",") + "]")

In [16]:
type(annonces)

list

In [17]:
type(annonces[0])

dict

In [18]:
annonces[0]

{'id': 'annonce-138905473-376235',
 'genre': 'Appartement',
 'prix': '374 400 €',
 'pcs': '3 p 2 ch 90 m²',
 'desc': "Appartement type 3 - TOURS CATHÉDRALE TOURS CATHÉDRALE: Ravissant appartement type 3 d'environ 90 m² dans résidence de standing (~50 lots) avec cave et parking. Entrée,...\nÇa m'intéresse",
 'lien': 'https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm'}

In [19]:
len(annonces)

1818

# Extraire l'information pertinente des colonnes

e.g. 
- id (str), 
- genre (catégorie), 
- prix (int), 
- surface (int), 
- pieces (int), 
- neuf (bool), 
- quartier (categorie?)
- ?

In [20]:
import pandas as pd

In [42]:
df = pd.DataFrame(annonces)

In [43]:
df.describe()

,id,genre,prix,pcs,desc,lien
count,1818,1818,1818,1818,1818,1818
unique,1798,16,1152,1140,1591,1798
top,annonce-140014391-548829,Appartement,155 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.seloger.com/annonces/investissemen...
freq,2,1041,11,30,19,2


In [44]:
df.head()

,id,genre,prix,pcs,desc,lien
0,annonce-138905473-376235,Appartement,374 400 €,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...
1,annonce-140620177-376235,Appartement,499 200 €,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...
2,annonce-140620179-376235,Appartement,499 200 €,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...
3,annonce-133494153-376235,Maison / Villa,508 000 €,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...
4,annonce-137425993-376235,Maison / Villa,676 000 €,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...


In [45]:
df.dtypes

id       object
genre    object
prix     object
pcs      object
desc     object
lien     object
dtype: object

## Appartement/Maison

In [46]:
df.genre.unique()

array(['Appartement', 'Maison / Villa', 'Appartement neuf',
       'Projet de construction', 'Maison / Villa neuve', 'Bâtiment',
       'Parking', 'Bureau', 'Local commercial', 'Immeuble', 'Boutique',
       'Terrain', 'Divers', 'Loft/Atelier/Surface', 'Château',
       'Hôtel particulier'], dtype=object)

In [47]:
equivalent_appartements = {'Appartement', 'Appartement neuf'}
equivalent_maisons = {'Maison / Villa', 'Maison / Villa neuve', 'Château', 'Hôtel particulier'}

In [48]:
df["appartement"] = df.eval("genre in @equivalent_appartements")

In [49]:
df["maison"] = df.eval("genre in @equivalent_maisons")

In [50]:
df["a_supprimer"] = df.eval("not (maison | appartement)")

In [51]:
df.describe()

,id,genre,prix,pcs,desc,lien,appartement,maison,a_supprimer
count,1818,1818,1818,1818,1818,1818,1818,1818,1818
unique,1798,16,1152,1140,1591,1798,2,2,2
top,annonce-140014391-548829,Appartement,155 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.seloger.com/annonces/investissemen...,True,False,False
freq,2,1041,11,30,19,2,1253,1406,1665


## Neuf / Ancien

In [52]:
equivalent_neufs = {'Appartement neuf', 'Maison / Villa neuve'}
df["neuf"] = df.eval("genre in @equivalent_neufs")

## Prix

In [53]:
masque = df["prix"].str.replace("€", "").str.replace(" ", "").str.replace("HT", "").str.replace("HH", "").str.isdecimal()

In [54]:
df["prix"].loc[masque == False]

843    €
Name: prix, dtype: object

In [55]:
df.drop(843, inplace=True)

In [61]:
df["prix_nettoye"] = (
    df["prix"]
    .str.replace("€", "")
    .str.replace(" ", "")
    .str.replace("HT", "")
    .str.replace("HH", "")
    .astype(int)
)

In [57]:
df.columns

Index(['id', 'genre', 'prix', 'pcs', 'desc', 'lien', 'appartement', 'maison',
       'a_supprimer', 'neuf', 'prix_nettoye'],
      dtype='object')

In [58]:
df.dtypes

id              object
genre           object
prix            object
pcs             object
desc            object
lien            object
appartement       bool
maison            bool
a_supprimer       bool
neuf              bool
prix_nettoye     int32
dtype: object

## Nombre de pièces

In [60]:
df.pcs.sample(n=20)

1575    1 p 30,83 m² 1 tess
1638         4 p 2 ch 83 m²
85         5 p 3 ch 88,2 m²
1579      3 p 2 ch 73,21 m²
805          4 p 3 ch 82 m²
681          4 p 3 ch 76 m²
618        2 p 1 ch 43,1 m²
236          3 p 2 ch 78 m²
97          8 p 4 ch 310 m²
1734       4 p 3 ch 74,2 m²
44          5 p 3 ch 110 m²
1386        7 p 6 ch 199 m²
991          4 p 2 ch 73 m²
1574        1 p 29 m² 5 etg
257       2 p 1 ch 48,87 m²
1008         3 p 2 ch 73 m²
279       4 p 3 ch 77,92 m²
1319        6 p 4 ch 170 m²
388         7 p 6 ch 182 m²
841        4 p 96 m² 1 park
Name: pcs, dtype: object

### Elaboration d'un motif

In [63]:
import re

In [129]:
motif1 = re.compile("([1-9][0-9]*)\\s*p\\s*([1-9][0-9]*)?\\s*c?h?\\s*([1-9][0-9]*),?([0-9]+)?\\s*m².*")

In [130]:
motif1.match("4 p 3 ch 77,92 m²")

<re.Match object; span=(0, 17), match='4 p 3 ch 77,92 m²'>

In [131]:
motif1.match("4 p 3 ch 77,92 m²").groups()

('4', '3', '77', '92')

In [132]:
motif1.match("7 p 6 ch 199 m²")

<re.Match object; span=(0, 15), match='7 p 6 ch 199 m²'>

In [133]:
motif1.match("7 p 6 ch 199 m²").groups()

('7', '6', '199', None)

In [134]:
motif1.match("2 p 39,89 m² 1 asc")

<re.Match object; span=(0, 18), match='2 p 39,89 m² 1 asc'>

In [135]:
df.pcs.str.match(motif1).value_counts()

True     1667
False     150
Name: pcs, dtype: int64

In [136]:
df.pcs.str.match(motif1)[df.a_supprimer].value_counts()

False    143
True      10
Name: pcs, dtype: int64

In [137]:
masque = ~df.a_supprimer & ~df.pcs.str.match(motif1)

In [139]:
df.pcs[masque]

430    2 p 1 ch 1 etg
434    4 p 3 ch 1 etg
436          9 p 4 ch
511               2 p
616    3 p 1 ch 1 asc
617    4 p 3 ch 3 etg
619         1 p 3 etg
Name: pcs, dtype: object

In [143]:
df["a_supprimer"] = df["a_supprimer"] | masque

### Extraction des données

In [146]:
groupes = df.pcs.str.extract(motif1)

In [152]:
groupes.head()

,0,1,2,3
0,3,2,90,NaN
1,5,4,146,27
2,5,3,110,NaN
3,6,4,132,NaN
4,7,5,185,NaN


In [149]:
df["nombre_pieces"] = groupes[0].astype(float)

In [153]:
df["surface"] = groupes[2].astype(float)

## Elimination des entrée inutiles

In [162]:
df.columns

Index(['id', 'genre', 'prix', 'pcs', 'desc', 'lien', 'appartement', 'maison',
       'a_supprimer', 'neuf', 'prix_nettoye', 'nombre_pieces', 'surface'],
      dtype='object')

In [211]:
filtre = (
    df
    .query("not a_supprimer")
    .drop(columns=["genre", "prix", "pcs", "a_supprimer"])
    .drop_duplicates()
    .drop(columns="id")
)

# Exploitation de la colonne lien

In [212]:
filtre.columns

Index(['desc', 'lien', 'appartement', 'maison', 'neuf', 'prix_nettoye',
       'nombre_pieces', 'surface'],
      dtype='object')

In [213]:
filtre["lien"] = (
    filtre.lien
    .str.replace("https://www.seloger.com/annonces/", "")
    .str.replace("https://www.selogerneuf.com/annonces/", "")
)

In [214]:
filtre.lien.str.split("/")

0       [achat, appartement, tours-37, cathedrale, 138...
1       [achat, appartement, tours-37, centre, 1406201...
2       [achat, appartement, tours-37, centre, 1406201...
3        [achat, maison, tours-37, centre, 133494153.htm]
4       [achat-de-prestige, maison, tours-37, centre, ...
                              ...                        
1807    [achat, appartement, tours-37, 134769693, #?cm...
1812        [achat, appartement, tours-37, 123293873.htm]
1813        [achat, appartement, tours-37, 141342739.htm]
1814    [achat, maison, tours-37, 137994967, #?cmp=INT...
1815    [achat, maison, tours-37, 138636083, #?cmp=INT...
Name: lien, Length: 1643, dtype: object

In [215]:
filtre.lien.str.split("/").apply(len)

0       5
1       5
2       5
3       5
4       5
       ..
1807    5
1812    4
1813    4
1814    5
1815    5
Name: lien, Length: 1643, dtype: int64

In [216]:
filtre.lien.str.split("/").apply(lambda ls: ls[0]).value_counts()

achat                1242
investissement        231
achat-de-prestige     115
https:                 55
Name: lien, dtype: int64

In [217]:
compte = filtre.lien.str.split("/").apply(lambda ls: ls[-2]).value_counts()
compte

tours-37            866
saint-symphorien     73
centre               70
grammont             59
europe               36
                   ... 
129996783             1
131904129             1
141452757             1
129123079             1
136292397             1
Name: lien, Length: 241, dtype: int64

In [218]:
masque = (compte < 866) & (compte > 1)
masque

tours-37            False
saint-symphorien     True
centre               True
grammont             True
europe               True
                    ...  
129996783           False
131904129           False
141452757           False
129123079           False
136292397           False
Name: lien, Length: 241, dtype: bool

In [219]:
masque[masque].index

Index(['saint-symphorien', 'centre', 'grammont', 'europe', 'la-fuye-velpeau',
       'sainte-radegonde', 'les-fontaines', 'montjoyeux', 'rives-du-cher',
       'beaujardin', 'sanitas-rotonde', 'febvotte-marat', 'paul-bert',
       'cathedrale', 'lakanal-strasbourg', 'rabelais-tonnelle',
       'deux-lions-gloriette', 'lamartine', 'rochepinard', 'giraudeau',
       'la-bergeonnerie', 'douets-milletiere'],
      dtype='object')

In [220]:
quartiers = list(masque[masque].index)

In [221]:
quartiers

['saint-symphorien',
 'centre',
 'grammont',
 'europe',
 'la-fuye-velpeau',
 'sainte-radegonde',
 'les-fontaines',
 'montjoyeux',
 'rives-du-cher',
 'beaujardin',
 'sanitas-rotonde',
 'febvotte-marat',
 'paul-bert',
 'cathedrale',
 'lakanal-strasbourg',
 'rabelais-tonnelle',
 'deux-lions-gloriette',
 'lamartine',
 'rochepinard',
 'giraudeau',
 'la-bergeonnerie',
 'douets-milletiere']

In [223]:
def choisit_quartier(lien):
    for quartier in quartiers:
        if quartier in lien:
            return quartier
    return "absence_de_renseignement"

In [224]:
filtre["quartier"] = filtre.lien.apply(choisit_quartier)

In [226]:
filtre.quartier.value_counts()

absence_de_renseignement    1084
saint-symphorien              73
centre                        70
grammont                      59
europe                        36
la-fuye-velpeau               33
sainte-radegonde              32
les-fontaines                 28
montjoyeux                    27
rives-du-cher                 23
beaujardin                    21
sanitas-rotonde               20
paul-bert                     20
febvotte-marat                20
cathedrale                    18
lakanal-strasbourg            14
rabelais-tonnelle             14
deux-lions-gloriette          13
lamartine                     11
rochepinard                   10
giraudeau                      9
douets-milletiere              4
la-bergeonnerie                4
Name: quartier, dtype: int64

# Exploitation de la colonne desc

In [227]:
filtre.sample(20).desc

1676    UN NOUVEAU QUARTIER, UNE NOUVELLE VIE - Idéale...
254     IAD France - Aurélien LE DEAUT vous propose: Q...
1691    Appartement situé au deuxième étage d'une peti...
34      Située à proximité du CHUR et de la faculté de...
413     Un produit remarquable, une adresse idéale, vo...
983     Résidence Bellevue, Avenue de la République à ...
285     IAD France - Alban BRIE vous propose: Dans une...
1721    Tours Nord - Dans résidence de standing, appar...
1256    IAD France - Nicolas BOURIN vous propose: * * ...
341     IAD France - Cecile BUSSONNAIS vous propose: "...
8       TOURS PRÉBENDES NORD - EXCLUSIVITÉ TOURS PRÉBE...
1551    Réf. 1695 Maison de maître 20 kms Tours Belle ...
1504    A vendre Maison / villa TOURS (37000) TOURS, M...
637     Face à la Loire, Panoramic est une résidence é...
1500    Maison/appartement 3 chambres. Superbe restaur...
135     Idéalement situé au coeur de Tours, appartemen...
1777    TOURS HALLES, Au 4e étage avec ascenseur, appa...
809     A deux

In [229]:
table = dict()

In [230]:
for c in "éèëêE":
    table[ord(c)] = ord("e")

In [231]:
table

{233: 101, 232: 101, 235: 101, 234: 101, 69: 101}

In [232]:
"éEeëê".translate(table)

'eeeee'

In [234]:
filtre.desc.str.lower().str.translate(table)

0       appartement type 3 - tours cathedrale tours ca...
1       tours hypercentre - appartement tours hypercen...
2       tours prebendes nord - appartement tours prebe...
3       tours prebendes - particulier tourangeaux tour...
4       tours strasbourg / rabelais - maison tours str...
                              ...                        
1807    maisons de ville 3 chambres avec jardin ou app...
1812    tours - fontaines, appartement de type 2 compr...
1813    tours - appartement de type 3 comprenant sejou...
1814    maisons de ville 3 chambres avec jardin ou app...
1815    maisons de ville 3 chambres avec jardin ou app...
Name: desc, Length: 1643, dtype: object

In [236]:
re.sub("\\W+", " ", "le, - blabla.: ! 123 ()")

'le blabla 123 '

In [239]:
integrale = filtre.desc.str.lower().str.translate(table).str.replace("\\W+", " ").str.cat()

In [240]:
integrale[:100]

'appartement type 3 tours cathedrale tours cathedrale ravissant appartement type 3 d environ 90 m² da'

In [241]:
token = integrale.split(" ")

In [242]:
len(token)

48319

In [243]:
compteur = dict()
for mot in token:
    compteur[mot] = compteur.get(mot, 0) + 1

In [245]:
len(compteur)

2610

In [244]:
compteur

{'appartement': 852,
 'type': 255,
 '3': 265,
 'tours': 832,
 'cathedrale': 39,
 'ravissant': 7,
 'd': 753,
 'environ': 197,
 '90': 11,
 'm²': 616,
 'dans': 532,
 'residence': 449,
 'de': 2651,
 'standing': 86,
 '50': 23,
 'lots': 24,
 'avec': 1024,
 'cave': 75,
 'et': 1128,
 'parking': 64,
 'entree': 392,
 'ça': 1643,
 'm': 1674,
 'interessetours': 331,
 'hypercentre': 4,
 'superbe': 36,
 '146': 1,
 '37': 7,
 'loi': 20,
 'carrez': 30,
 'un': 727,
 'immeuble': 60,
 'caractere': 29,
 'ascenseur': 169,
 'sejour': 344,
 'sud': 136,
 'vaste': 46,
 'prebendes': 81,
 'nord': 229,
 'bel': 149,
 '110': 9,
 'petit': 29,
 '5': 94,
 'terrasse': 100,
 '56': 6,
 '36': 5,
 'particulier': 49,
 'tourangeaux': 4,
 'agreable': 80,
 'maison': 313,
 'particuliere': 9,
 '132': 1,
 'jardin': 121,
 'expose': 67,
 '130': 6,
 'salon': 239,
 'salle': 242,
 'à': 839,
 'manger': 39,
 'strasbourg': 15,
 'rabelais': 24,
 'confortable': 9,
 'recente': 64,
 '185': 1,
 'edifiee': 10,
 'sur': 321,
 'terrain': 30,
 '500